In [1]:
from src.models.Encodec import Encodec
encodec = Encodec()


/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [2]:
from audiocraft.modules.codebooks_patterns import DelayedPatternProvider

pattern_provider = DelayedPatternProvider(4)

import torch
test = torch.randn(1,24000*20)

test2 = {"wav" : torch.randn(2,1,24000*20),
        "original_lens" : torch.Tensor([[12000],[12000]]).int()}

codes = encodec(test)


In [9]:
from src.models.model import MuMRVQ
from src.models.Transformer.Encoder import VanillaEncoder
from src.models.Transformer.Decoder import VanillaDecoder
model = MuMRVQ(VanillaEncoder(layers=2), VanillaDecoder(layers=2) ,encodec, debug = False, pattern="none", mask_before=True)

In [10]:
out_ = model(test2)

tensor(37, dtype=torch.int32)
tensor(37, dtype=torch.int32)
codes: torch.Size([2, 4, 1500])
padding_mask: torch.Size([2, 1501])
codes: torch.Size([2, 4, 1024])
padding_mask: torch.Size([2, 1024])
masking before with masking proba : 0.5
============== codes_masking ==============
torch.Size([2, 4, 1024])
4118 tokens were masked with random masking
shape coming out of encoder: ============
torch.Size([2, 1025, 960])
========= retained idx shape ============
torch.Size([2, 1024])
=========== Masked without embeddings shape ========
torch.Size([2, 1024, 960])
========= all_masked.shape ==========
torch.Size([2, 1024, 960])
========= around masked index ========
tensor([[ 0.7141,  0.0028, -0.0033],
        [-0.2952,  0.1390, -0.1742]])
tensor([[-0.9959,  1.1102,  0.7493],
        [ 0.2182, -0.4436, -0.0730]])
class token =============
torch.Size([2, 1, 960])
========= All outputs for the encoder========
-------- masked output with class token --------
torch.Size([2, 1025, 960])
-------- unm

In [ ]:
out_['codes'].shape

torch.Size([2, 4, 1024])

In [ ]:
from torch.nn import CrossEntropyLoss

CrossEntropyLoss(ignore_index=1026)(out_['logits'][:,:,:,1:],out_['codes'])

RuntimeError: size mismatch (got input: [2, 1027, 4, 512] , target: [2, 4, 1024]

In [ ]:
out_['codes']

tensor([[[1025,  732, 1025,  ..., 1025, 1026, 1026],
         [ 691,  691, 1025,  ..., 1025, 1026, 1025],
         [1025,  919, 1025,  ..., 1026, 1026, 1025],
         [1025, 1025,  988,  ..., 1026, 1026, 1026]],

        [[1025,  732, 1025,  ..., 1026, 1025, 1025],
         [ 691, 1008,  662,  ..., 1026, 1026, 1026],
         [1025, 1025,  186,  ..., 1026, 1025, 1026],
         [1025, 1025, 1025,  ..., 1025, 1026, 1026]]])

In [ ]:
test = torch.softmax(torch.Tensor([1,-float('inf')]),dim=0)